In [1]:
pip install textblob

^C
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install syllables

In [1]:
import pandas as pd
from textblob import TextBlob
import nltk
import syllables
from bs4 import BeautifulSoup
import requests
import pandas as pd

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Prakalp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Prakalp\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
def calculate_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity, blob.sentiment.subjectivity

In [3]:
def calculate_sentence_stats(text):
    sentences = nltk.sent_tokenize(text)
    words_per_sentence = [len(nltk.word_tokenize(sentence)) for sentence in sentences]

    avg_sentence_length = sum(words_per_sentence) / len(words_per_sentence)
    avg_words_per_sentence = sum(words_per_sentence) / len(sentences)

    return avg_sentence_length, avg_words_per_sentence

In [4]:
def calculate_complex_word_stats(text, complex_words):
    words = nltk.word_tokenize(text)
    complex_word_count = sum(word in complex_words for word in words)
    percentage_complex_words = (complex_word_count / len(words)) * 100

    return percentage_complex_words, complex_word_count

In [5]:
def calculate_fog_index(avg_sentence_length, percentage_complex_words):
    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)
    return fog_index

In [6]:
def calculate_syllables_per_word(text):
    words = nltk.word_tokenize(text)
    syllables_per_word = sum([syllables.estimate(word) for word in words]) / len(words)
    return syllables_per_word

In [7]:
def count_personal_pronouns(text, personal_pronouns):
    words = nltk.word_tokenize(text)
    personal_pronoun_count = sum(word.lower() in personal_pronouns for word in words)
    return personal_pronoun_count

In [8]:
def calculate_avg_word_length(text):
    words = nltk.word_tokenize(text)
    avg_word_length = sum(len(word) for word in words) / len(words)
    return avg_word_length

In [11]:
df = pd.read_excel('input.xlsx')
urls = df['URL']
url_ids = df['URL_ID']

complex_words = ['abject', 'aberration', 'abnegation']

personal_pronouns = ['I', 'me', 'my', 'you', 'your']

output_data = pd.DataFrame(columns=['URL_ID', 'URL', 'POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE',
                                    'SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH', 'PERCENTAGE OF COMPLEX WORDS',
                                    'FOG INDEX', 'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT',
                                    'WORD COUNT', 'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH'])

In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from textblob import TextBlob
import nltk


output_data = pd.DataFrame(columns=['URL_ID', 'URL', 'POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE',
                                    'SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH', 'PERCENTAGE OF COMPLEX WORDS',
                                    'FOG INDEX', 'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT',
                                    'WORD COUNT', 'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH'])

for url, url_id in zip(urls, url_ids):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        title_element = soup.find('title')
        article_text_element = soup.find('div', {'class': 'td-pb-span8 td-main-content'})

        title = title_element.get_text() if title_element else "Title not found"
        article_text = article_text_element.get_text() if article_text_element else "Article text not found"

        filename = f"{url_id}.txt"
        with open(filename, 'w', encoding='utf-8') as file:
            file.write(f"{title}\n\n{article_text}")

        print("File created")

        positive_score, subjectivity_score = calculate_sentiment(article_text)
        negative_score = 1 - positive_score
        avg_sentence_length, avg_words_per_sentence = calculate_sentence_stats(article_text)
        percentage_complex_words, complex_word_count = calculate_complex_word_stats(article_text, complex_words)
        fog_index = calculate_fog_index(avg_sentence_length, percentage_complex_words)
        syllables_per_word = calculate_syllables_per_word(article_text)
        personal_pronoun_count = count_personal_pronouns(article_text, personal_pronouns)
        avg_word_length = calculate_avg_word_length(article_text)

        output_data_row = pd.DataFrame({
            'URL_ID': [url_id],
            'URL': [url],
            'POSITIVE SCORE': [positive_score],
            'NEGATIVE SCORE': [negative_score],
            'POLARITY SCORE': [positive_score - negative_score],
            'SUBJECTIVITY SCORE': [subjectivity_score],
            'AVG SENTENCE LENGTH': [avg_sentence_length],
            'PERCENTAGE OF COMPLEX WORDS': [percentage_complex_words],
            'FOG INDEX': [fog_index],
            'AVG NUMBER OF WORDS PER SENTENCE': [avg_words_per_sentence],
            'COMPLEX WORD COUNT': [complex_word_count],
            'WORD COUNT': [len(nltk.word_tokenize(article_text))],
            'SYLLABLE PER WORD': [syllables_per_word],
            'PERSONAL PRONOUNS': [personal_pronoun_count],
            'AVG WORD LENGTH': [avg_word_length]
        })

        output_data = pd.concat([output_data, output_data_row], ignore_index=True)

    except Exception as e:
        print(f"Error processing URL {url}: {e}")

output_data.to_excel('output_data.xlsx', index=False)


File created
File created
File created
File created
File created
File created
File created
File created
File created
